In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

# 数据预处理

In [1]:
df_train.drop(['Name','Ticket'], axis = 1,inplace=True)
df_test.drop(['Name','Ticket'], axis = 1,inplace=True)

df_train['Age'].fillna(value=29,inplace=True)
df_test['Age'].fillna(value=29,inplace=True)

df_train['Embarked'].fillna(value='S',inplace=True)	
df_test['Embarked'].fillna(value='S',inplace=True)

df_train.Cabin.fillna(value='N',inplace=True)
df_train.Cabin = df_train.Cabin.str[:1]

df_test.Cabin.fillna(value='N',inplace=True)
df_test.Cabin = df_train.Cabin.str[:1]

df_test.Fare.fillna(value=32.2,inplace=True)

df_train['Pclass'] = df_train['Pclass'].astype(str)	
df_test['Pclass'] = df_test['Pclass'].astype(str)

In [1]:
int_col = ['Age', 'SibSp', 'Parch','Fare']

df_mean = df_train[int_col].mean(axis=0)
df_std = df_train[int_col].std(axis=0)

df_test[int_col] = (df_test[int_col] - df_mean) / df_std
df_train[int_col] = (df_train[int_col] - df_mean) / df_std

df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [1]:
col_X = ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_N',
       'Cabin_T', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
col_y = ['Survived']

# 模型

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers

In [1]:
train_1,train_2 = train_test_split(df_train,test_size=0.25)   # 将数据分为75%训练集和25%测试集

In [1]:
mo_1 = models.Sequential()
mo_1.add(layers.Dense(32, activation='relu', input_shape=(21,)))
mo_1.add(layers.Dense(16, activation='relu'))
mo_1.add(layers.Dense(4, activation='relu'))
mo_1.add(layers.Dense(1, activation='sigmoid'))

In [1]:
mo_1.compile(optimizer='rmsprop',          # 优化器
              loss='binary_crossentropy',   # 损失函数
              metrics=['binary_accuracy'])         # 评估函数

In [1]:
history_1 = mo_1.fit(train_1[col_X],    # 训练集X
                    train_1[col_y],    # 训练标签
                    epochs=64,          # 迭代次数
                    batch_size=128,     # 梯度下降组
                    validation_data=(train_2[col_X], train_2[col_y])) # 验证集

In [1]:
hh = pd.DataFrame( mo_1.history.history )
hh.plot.line()

In [1]:
mo_2 = models.Sequential()
mo_2.add(layers.Dense(32, activation='relu', input_shape=(21,)))
mo_2.add(layers.Dense(4, activation='relu'))
mo_2.add(layers.Dense(1, activation='sigmoid'))

mo_2.compile(optimizer='rmsprop',          # 优化器
              loss='binary_crossentropy',   # 损失函数
              metrics=['binary_accuracy'])         # 评估函数

history_1 = mo_2.fit(df_train[col_X],    # 训练集X
                    df_train[col_y],    # 训练标签
                    epochs=255,          
                    batch_size=128,verbose=0) 

In [1]:
df_test['Survived'] = mo_2.predict(df_test[col_X])
df_test.loc[df_test['Survived']>=0.5,'Survived'] = 1
df_test.loc[df_test['Survived']< 0.5,'Survived'] = 0
df_test['Survived'] = df_test['Survived'].astype('int')
df_test[ ['PassengerId','Survived'] ].to_csv('submission.csv',index=False)